# Modelagem de tópicos com CTM/ETM/LDA no *Reddit* em português

Os testes a seguir foram realizados com o *corpus* em português de submissões do *Reddit* relacionadas à depressão. Deve-se salientar que apenas a remoção de termos com número pequeno de ocorrências e a tokenização foram aplicadas como etapas de pré-processamento do *dataset*. Para o treinamento ETM, o embedding Word2Vec utilizado foi criado usando a API de Word2Vec disponibilizada pelo próprio gensim.

Além disso, para remoção de termos baseado em número de ocorrências, foram usados os valores de min_df = 0.05 e de max_df = 0.75 nos vetorizadores de texto utilizados no pré-processamento do referido *corpus*. Com tais valores, o conjunto de termos no *corpus* foi reduzido significativamente.

### Importações e configurações de ambiente

In [19]:
%load_ext autoreload
%autoreload 2

from utils.plots import plot_wordcloud, plot_graph, plot_wordcloud_by_word_probability, plot_pie_chart
import sys, time, json, os, joblib, numpy as np, pandas as pd, ast
pd.options.display.max_colwidth = 100
from utils.notebook import create_dataframe_from_topics, get_topic_word_probability_pairs
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 8]

pyLDAvis.enable_notebook()

WORKDIR = os.getcwd()
MODELS_PATH = f'{WORKDIR}/models'
CSVS_PATH = f'{WORKDIR}/csvs/'
RESOURCES_PATH = f'{WORKDIR}/resources/'
CTM_MODELS_PATH = f'{MODELS_PATH}/ctm/'
ETM_MODELS_PATH = f'{MODELS_PATH}/etm/'
LDA_MODELS_PATH = f'{MODELS_PATH}/lda/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Resultados do treinamento CTM

In [20]:
ctm_results = pd.read_csv(CSVS_PATH + "ctm_combined_results.csv")

In [21]:
results_by_coherence = ctm_results.sort_values(["c_npmi"], ascending=(False))
results_by_coherence.head()

,Unnamed: 0,k,model,c_v,u_mass,c_uci,c_npmi,diversity,path
0,0,5.0,ctm_k5_combined,0.338887,-2.109887,-1.17098,-0.039418,0.53,training_outputs/models/ctm/ctm_k5_combined


In [22]:
ctm1 = joblib.load(CTM_MODELS_PATH + "ctm_k5_combined")

Tópicos encontrados pelo modelo CTM melhor ranqueado:

In [23]:
for idx, topic in enumerate(ctm1["topics"]):
    print(f'Topico {idx}: {" ".join(topic)}\n')

Topico 0: prazer pessoal foder video tardar curso energia respeitar aguentar ultimar certeza estudo sortir contrariar redor usar vaziar livro alar noite

Topico 1: vidar casar pessoa coisa filho familia epoca mundo mulher historiar filhar irma homem tempo cidade gente estar trabalhar lugar irmao

Topico 2: pessoa tempo coisa amigo sentimento amizade garoto amigar relacao converso mensagem gente namorar relacionamento final problema vezar lado frente sentir

Topico 3: empregar estagiar faculdade cursar situacao crise area salariar dinheiro problema semana trabalhar empresar ansiedade curso tratamento aula contar casar provar

Topico 4: ultimar curso foder video prazer tardar respeitar variar contrariar estudo conheco pessoal alar livro aguentar sortir encontrar menino usar redor



Tópico 0: <br>
1: emprego / trabalho<br>
2: <br>
3: <br>
4: <br>
5: <br>
6: <br>
7: <br>
8: <br>
9: <br>
10: rotina na quarentena<br>
11: política<br>
12: <br>
13: doença mental<br>
14: ruminação<br>
15: ruminação<br>
16: <br>
17: abuso infantil<br>
18: <br>
19: diagnóstico / tratamento<br>
20: ruminação / pensamentos negativos<br>
21: relacionamento<br>
22: <br>
23: <br>
24: relacionamento / amizade<br>
25: <br>
26: <br>
27: estudos<br>

### Resultados de treinamento ETM

In [24]:
etm_results = pd.read_csv(CSVS_PATH + "etm_results.csv")
etm_results.sort_values(["c_npmi"], ascending=(False))

,Unnamed: 0,k,model,c_v,u_mass,c_uci,c_npmi,diversity,path
0,0,5.0,etm_k5,0.424796,-1.556681,0.00329,0.000643,0.55,training_outputs/models/etm/etm_k5


In [25]:
emt1 = joblib.load(ETM_MODELS_PATH + "etm_k5")

In [26]:
for idx, topic in enumerate(emt1['topics']):
    print(f'Topico {idx}: {" ".join(topic)}\n')

Topico 0: amigo pessoa coisa tempo gente relacionamento namorar vezar caro garoto amizade entao amigar converso tipo mensagem semana lado menino partir

Topico 1: problema ansiedade coisa tempo formar crise vidar ajudar causar psiquiatro vontade voces tipo motivar tratamento terapia sentir atar situacao variar

Topico 2: faculdade trabalhar escola vidar cursar tempo pessoa empregar dinheiro coisa partir aula cidade formar lugar ensinar atar professorar sociedade empresar

Topico 3: pessoa vidar coisa vezar mundo sentir medo suicidio tempo momento vontade amor morte verdade sentimento mentir pensamento fato tristeza atar

Topico 4: casar familia vidar coisa mulher semana entao tempo noite pessoa atar filho remedios caro homem horar suicidio vezar lugar irma



Tópico 0: <br>
1: <br>
2: <br>
3: <br>
4: <br>
5: <br>
6: relacionamentos<br>
7: <br>
8: <br>
9: pensamentos mórbidos<br>
10: <br>
11: <br>
12: <br>
13: tratamento de depressão<br>
14: família<br>
15: ruminação<br>
16: estudos / trabalho<br>
17: relacionamentos/familia<br>

### Resultados de treinamento LDA

In [27]:
lda_results = pd.read_csv(CSVS_PATH + "lda_results.csv")
lda_results.sort_values(["c_npmi"], ascending=(False))

,Unnamed: 0,k,model,c_v,u_mass,c_uci,c_npmi,diversity,path
0,0,5.0,lda_k5,0.217436,-2.783523,-2.504574,-0.08926,0.5,training_outputs/models/lda/lda_k5


In [28]:
lda1 = joblib.load(LDA_MODELS_PATH + "lda_k5")
for idx, topic in enumerate(lda1["topics"]):
    print(f'Topico {idx}: {" ".join(topic)}\n')

Topico 0: salariar rotina empresar carreira fracassar bolsar area filho estagiar universidade curso panico semestre jogar roupar medicar quarentenar remedio cursar grupar

Topico 1: voce roupar salariar mensagem semestre carreira quarentenar irma erro estagiar maximo madrugar rede atitude foto ficar desculpar solidao idade super

Topico 2: drogar estagiar salariar universidade empresar bolsar cursar energia solucao estudar redor realidade filho vaziar remedio roupar usar medicar ruir homem

Topico 3: universidade estagiar semestre salariar professorar solucao carreira perspectivo sucesso sofrer tristeza tentar estudo necessidade erro solidao esperanca deus valor noticiar

Topico 4: sintoma chorar drogar panico efeito erro fase sacar porra homem deixar solucao sexo perder ligar vaziar todo remedios quartar amizade



Tópico 0: sociedade<br>
1: relacionamento amoroso<br>
2: ensino<br>
3: emoções<br>
4: estudos<br>

### Resultados de treinamento gerais

Unificando os resultados em CSV avaliados anteriormente, pode-se determinar o modelo com maior valor de coerência dentre todos aqueles treinados (LDA, CTM, ETM).

In [29]:
df_geral = pd.concat([ctm_results, etm_results, lda_results], ignore_index=True)
df_geral.sort_values(["c_npmi"], ascending=(False))

,Unnamed: 0,k,model,c_v,u_mass,c_uci,c_npmi,diversity,path
1,0,5.0,etm_k5,0.424796,-1.556681,0.003290,0.000643,0.55,training_outputs/models/etm/etm_k5
0,0,5.0,ctm_k5_combined,0.338887,-2.109887,-1.170980,-0.039418,0.53,training_outputs/models/ctm/ctm_k5_combined
2,0,5.0,lda_k5,0.217436,-2.783523,-2.504574,-0.089260,0.50,training_outputs/models/lda/lda_k5


De forma geral os tópicos obtidos nesse relatório têm grandes deficiências, onde muitos dos tópicos encontrados apresentam termos com pouca relação entre si. Pode-se destacar a presença grande de termos inúteis nos tópicos LDA (especialmente a palabra "removed").